1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

Описание данных - https://archive.ics.uci.edu/ml/datasets/banknote+authentication#

In [79]:
import pandas as pd
import numpy as np
df = pd.read_csv("data_banknote_authentication.txt", header=None)
df.head(3)

,0,1,2,3,4
0,3.6216,8.6661,-2.8073,-0.44699,0
1,4.5459,8.1674,-2.4586,-1.46210,0
2,3.8660,-2.6383,1.9242,0.10645,0


In [80]:
df.columns=['Variance', 'Skewness', 'Curtosis', 
              'Entropy','Class']
data.head(3)

,variance,skewness,curtosis,entropy,class
0,3.6216,8.6661,-2.8073,-0.44699,0
1,4.5459,8.1674,-2.4586,-1.46210,0
2,3.8660,-2.6383,1.9242,0.10645,0


In [81]:
df['Class'].value_counts()

0    762
1    610
Name: Class, dtype: int64

Проверим метрики

### Обучим LogisticRegression

In [82]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=3)

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [83]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    return [f1,roc,rec,prc]

result= evaluate_results(y_test, y_predict)
result.append('step4')
A = pd.DataFrame([result],columns=['f1', 'roc', 'recall', 'precision','Model'])

Classification results:
f1: 98.32%
roc: 98.73%
recall: 100.00%
precision: 96.69%


### далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

In [84]:
share = .05

mod_data = df.copy()
#get the indices of the positives samples
P = np.where(mod_data['Class'].values == 1)[0]
#shuffle them
np.random.shuffle(P)
# leave just share% of the positives marked
P_len = int(np.ceil(share * len(P)))
print(f'Using {P_len}/{len(P)} as positives and unlabeling the rest')
P_sample = P[:P_len]

Using 31/610 as positives and unlabeling the rest


In [85]:
# столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
mod_data['class_test'] = -1
mod_data.loc[P_sample,'class_test'] = 1
print('target variable:\n', mod_data['class_test'].value_counts())

target variable:
 -1    1341
 1      31
Name: class_test, dtype: int64


31 значение отмечено как положительные, а остальные - отрицательные.

In [86]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.class_test.values # new class (just the P & U)
y_positive = mod_data.Class.values # original class

### применить random negative sampling для построения классификатора в новых условиях

In [87]:
mod_data = mod_data.sample(frac=1)

neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(31, 6) (31, 6)


In [90]:
model = LogisticRegression(random_state=3)

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

result= evaluate_results(sample_test.iloc[:,-2].values, y_predict)
result.append('last step')
B = pd.DataFrame([result],columns=['f1', 'roc', 'recall', 'precision','Model'])

Classification results:
f1: 95.34%
roc: 96.32%
recall: 100.00%
precision: 91.10%


In [91]:
AB = A.append(B)
AB = AB[['Model','f1', 'roc', 'recall', 'precision']]
AB

,Model,f1,roc,recall,precision
0,step4,0.983193,0.987342,1.0,0.966942
0,last step,0.953429,0.963186,1.0,0.911003


### поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

Если менять незначительно, то и изменения минимальны. А вот если сильно увеличить P то метрики становятся лучше и наоборот